In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Avvalgi amaliyotda `GitHub`ga yuklangan `melb_data.csv` faylni `df` ga chaqirib oling

In [48]:
df = pd.read_csv('https://raw.githubusercontent.com/eldorjonneymatov/mdev_ds_pract/main/data_cleaning/melb_data.csv?token=GHSAT0AAAAAABWHQ7BADJMCPRPSIEVH6XMIYWQEPYQ')
df.head()

Yuqoridagi Dataframe Melburn shahri uchun sotuvdagi uylar haqida m'lumotlarni jamlagan.

Quyida jadvalning ba'zi sutunlari tarjimasi berilgan (qolganini o'zingiz Google yordamida tarjima qilishingiz mumkin):
- `Room` - xonalar soni
- `Type` - uyning turi (h - villa/kottej/hovli; u - kvartira; t - shahardagi hovli)
- `Price` - narx
- `Bathroom` - tualet-vanna
- `Landsize` - yerning maydoni (hovli uchun)
- `BuildingArea` - Bino qurilgan yerning maydoni
- `YearBuilt` - qurilgan yili
- `Lattitude` va `Longitude` - GPS kordinatalar
- `Regionname` - Region

# 🔺 DIQQAT! 
#### **Agar alohida aytilmagan bo'lsa, `df`ga kiritilgan o'zgartirishlarni saqlamang (`inplace=False`)**

## 1-VAZIFA. `df` dagi barcha ustunlardagi`NaN` qiymatlar sonini chiqaring

In [74]:
#Javobingizni shu yerga yozing
df.isnull().sum()

## 2-VAZIFA. `df` barcha `NaN` qiymatlarni 0 bilan almashtiring

In [75]:
#Javobingizni shu yerga yozing
df.fillna(0)

## 3-VAZIFA. Quyidagi 2 ustun uchun o'rta qiymatlarni hisoblang:
- BuildingArea
- YearBuilt (**butun son qaytishi kerak**)

In [76]:
#Javobingizni shu yerga yozing
barea_m = df.BuildingArea.mean()
ybuilt_m = int(df.YearBuilt.mean())
means = pd.Series([barea_m, ybuilt_m], index = ['BuildingArea', 'YearBuilt'])
means

## 4-VAZIFA: `BuildingArea` va `YearBuilt` ustunlaridagi `NaN` qiymatlarni shu sutun uchun o'rta qiymatlar bilan almashtiring
**DIQQAT!** `YearBuilt` ustuni qiymatlari butun son bo'lishi kerak

In [77]:
#Javobingizni shu yerga yozing
df.fillna({'BuildingArea': barea_m, 'YearBuilt': ybuilt_m})

## 5-VAZIFA. `.groupby()`
Ustun uchun o'rta qiymat har doim ham aniq bo'lavermaydi. Keling avval `YearBuilt` ustuni uchun aniqroq qiymat topishga harakat qilamiz. 

Buning uchun `Regionname` ustuni qiymatlaridan foydalanishimiz mumkin, ya'ni bir Regiondagi binolar tahminan bir hil yillarda qurilgan deb tahmin qilsak bo'ladi.

### 5.1 `.groupby()` yordamida `df` qiymatlarini `Regionname` ustuni bo'yicha guruhlang va `YaerBuilt` ustuni uchun o'rta qiymatni hisoblang.

In [78]:
#Javobingizni shu yerga yozing
reg_mean = df.groupby('Regionname').YearBuilt.mean().astype(int)
reg_mean

### 5.2. `df` `YearBuilt` ustunidagi `NaN` qiymatlarni yuqoridagi qiymatlar bilan almashtiring.
`df[df.Regionname='Eastern Metropolitan']=...`

Har bir `Regionname` uchun qiymatlarni bittalab kiritmaslik uchun for siklidan foydalanishingiz mumkin.

In [79]:
#Javobingizni shu yerga yozing
for name in reg_mean.index:
    df[df.Regionname == name] = df[df.Regionname == name].fillna({'YearBuilt': reg_mean.loc[name]})
df.YearBuilt

### `YearBuilt` uchun yana ham aniqroq o'rta qiymat hisoblashning qanday yo'lini taklif qilgan bo'lar edingiz?

## 6-VAZIFA. `BuildingArea` ustunidagi `NaN` qiymatlarni to'ldiramiz
`BuildingArea` bino qurilgan yerning maydonini bildiradi. Bu qiymatlarni tahmin qilish uchun qaysi ustun qiymatlaridan foydalanish to'g'ri deb hisoblaysiz?

In [80]:
df.head()

### 6.1 `df`ni xonalar soni bo'yicha o'rta qiymat olib ko'ring

In [81]:
#Javobingizni shu yerga yozing
df.groupby('Rooms').BuildingArea.mean()

### 6.2 Xonalar va uy turi bo'yicha o'rta qiymatni olib ko'ring.

In [82]:
#Javobingizni shu yerga yozing
tr_mean = df.groupby(['Type', 'Rooms']).BuildingArea.mean()
tr_mean

### 6.3 Ahamiyat bering, 10 xonalik uylar uchun o'rta qiymat chiqmayapti. Bu nimadan dalolat beradi? Jami 10 xonalik uylar nechta? Ular bilan qanday yo'l tutishni maslahat berasiz?

In [83]:
#Javobingizni shu yerga yozing
df[df.Rooms == 10]

In [84]:
idx = df[df.Rooms == 10].index
df.drop(idx, inplace = True)

In [85]:
df[(df.Rooms == 8) & (df.Type == 'u')]

In [86]:
idx = df[(df.Rooms == 8) & (df.Type == 'u')].index
df.drop(idx, inplace = True)

### 6.4 `BuildingArea` ustunidagi mavjud `NaN` qiymatlarni o'zingiz to'g'ri deb bilgan usul bilan to'ldiring.

In [87]:
for h in tr_mean.index:
    df[(df.Type == h[0]) & (df.Rooms == h[1])] = df[(df.Type == h[0]) & (df.Rooms == h[1])].fillna({'BuildingArea': tr_mean.loc[h]})
df['BuildingArea']

### 7-VAZIFA. Yuqoridagi vazifalardan so'ng, bizda faqatgina `CouncilArea` ustunida `NaN` qiymatlar qolishi kerak xolos.
**Bu ustundagi `NaN` qiymatlarni avvalgi qatordagi qiymatlar bilan to'ldirib keting. Natijani saqlab qoling**

In [88]:
df['CouncilArea'] = df['CouncilArea'].fillna(method = 'ffill')

## YAKUNIY JADVAL
Ahamiyat bering, yakuniy jadvalda `NaN` qiymatlar bo'lmasligi kerak.

In [89]:
df.isnull().sum()